In [47]:
import pandas as pd
import math
import string
import re
import copy
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier  ## decision tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC

# Data

In [21]:
train_dataset = pd.read_csv('train_haralick.csv')
val_dataset = pd.read_csv('val_haralick.csv')
test_dataset = pd.read_csv('test_haralick.csv')

In [18]:
train_dataset

,id,B_std,G_std,R_std,B_mean,G_mean,R_mean,Angular Second Moment,Contrast,Correlation,...,Inverse Difference Moment,Sum Average,Sum Variance,Sum Entropy,Entropy,Difference Variance,Difference Entropy,f12,f13,label
0,n02111277_10005,130.345014,130.722372,119.028426,56.233264,51.611620,46.399578,0.001078,165.528623,0.967005,...,0.271132,254.197016,9864.984361,7.909656,11.601473,0.000386,4.117412,-0.330984,0.992478,0
1,n02111277_10019,100.212060,116.829706,108.123866,62.034469,76.657011,73.716812,0.000710,139.722019,0.986771,...,0.223512,224.384088,20986.941383,8.445584,12.288659,0.000282,4.310872,-0.355403,0.997330,0
2,n02111277_10072,119.017744,121.216963,120.036817,88.058211,88.287855,85.213120,0.010357,108.791055,0.992818,...,0.325182,240.764633,30198.035566,8.316080,11.619560,0.000374,4.059540,-0.432343,0.999106,0
3,n02111277_1008,160.950742,132.119066,117.695565,71.616624,53.815305,46.573737,0.000494,218.916002,0.955781,...,0.259032,262.097478,9679.433126,8.550477,12.414560,0.000308,4.442927,-0.366032,0.997648,0
4,n02111277_10089,130.182391,131.262303,128.459529,95.145334,95.822750,90.582937,0.006731,43.159091,0.997558,...,0.437478,260.052623,35305.078211,7.684574,10.251967,0.000604,3.363347,-0.488246,0.999270,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63320,n02172182_9515,88.512731,106.297148,134.053696,45.030752,47.632294,47.450994,0.000212,211.160415,0.951587,...,0.162520,225.176594,8512.843211,8.460733,12.907487,0.000216,4.681814,-0.275287,0.991113,49
63321,n02172182_954,53.953012,74.889609,82.939679,63.765287,69.598328,72.805093,0.000328,1027.361853,0.892529,...,0.143406,149.510709,18094.820693,8.494497,13.348533,0.000133,5.570192,-0.217942,0.979301,49
63322,n02172182_9549,95.058310,121.767588,123.922179,36.606144,42.809942,37.032092,0.000219,245.011683,0.914696,...,0.125613,238.613158,5498.868574,8.182608,12.900225,0.000173,4.900730,-0.212414,0.974447,49
63323,n02172182_9589,54.801239,118.761515,122.013855,37.805967,55.218147,59.685140,0.000333,117.674209,0.979277,...,0.215939,224.956447,11238.871999,8.491203,12.355053,0.000318,4.159846,-0.352856,0.997196,49


In [22]:
# train
Y_train = train_dataset['label']
X_train = train_dataset.copy()
del(X_train['label'])
del(X_train['id'])

# val
Y_val = val_dataset['label']
X_val = val_dataset.copy()
del(X_val['label'])
del(X_val['id'])

# test
Y_test = test_dataset['label']
X_test = test_dataset.copy()
del(X_test['label'])
del(X_test['id'])

# 定義model

In [61]:
# model1 XGBClassifier
model1 = XGBClassifier()
model1.fit(X_train,Y_train, eval_set=[(X_train, Y_train), (X_val, Y_val)], eval_metric='auc', early_stopping_rounds=10, verbose=True)
print('訓練集: ',model1.score(X_train,Y_train))

# model2 CatBoostClassifier
model2 = CatBoostClassifier(iterations=800,eval_metric = 'AUC', learning_rate=0.1 ,verbose=False)
model2.fit(X_train,Y_train, eval_set=[(X_train, Y_train), (X_val, Y_val)],plot=True)
print('訓練集: ',model2.score(X_train,Y_train))

# model3 SVM
model3 = SVC(probability=True)
model3.fit(X_train,Y_train)
print('訓練集: ',model3.score(X_train,Y_train))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

訓練集:  0.3188472167390446
測試集:  0.1511111111111111


# evaluated in top-1 accuracy and top-5 accuracy

In [83]:
def top_n_acc(pred_table,truth,n):
    acc = []
    f_n = len(pred_table.iloc[0])
    q = (1-(n/f_n))*100
    for i in range(len(pred_table)):
        target = pred_table.loc[i,pred_table.iloc[i,:]>np.percentile(pred_table.iloc[i,:],q)].index
        if truth.iloc[i][0] in target:
            acc.append(1)
        else:
            acc.append(0)
    output = sum(acc)/len(acc)
    return output

In [55]:
pre1 = model1.predict_proba(X_test)
pre2 = model2.predict_proba(X_test)
pre3 = model3.predict_proba(X_test)

In [81]:
truth = pd.DataFrame(Y_test)
pre1 = pd.DataFrame(pre1)
pre2 = pd.DataFrame(pre2)
pre3 = pd.DataFrame(pre3)

In [86]:
#
t_1_model1 = top_n_acc(pre1,truth,1)
t_1_model2 = top_n_acc(pre2,truth,1)
t_1_model3 = top_n_acc(pre3,truth,1)
t_5_model1 = top_n_acc(pre1,truth,5)
t_5_model2 = top_n_acc(pre2,truth,5)
t_5_model3 = top_n_acc(pre3,truth,5)

In [87]:
t_1_model1,t_1_model2,t_1_model3

(0.13111111111111112, 0.14222222222222222, 0.05555555555555555)

In [88]:
t_5_model1,t_5_model2,t_5_model3

(0.3888888888888889, 0.4266666666666667, 0.20666666666666667)